<a href="https://colab.research.google.com/github/ktichola/Image-matching/blob/main/IMC_2024_Multi_models_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-matching-challenge-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71885%2F8143495%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D13551573aa4a4cb329e5728667ea9e56c5def6e03687e4c3b68598ef3a0f632e21ac68fced5e32e8c63a72d68610e6434acce574e0492e883ef6640093f128cc9d448f110929dfd35aee695f7b53b920531b04e4b18addf3337e8eeee94528b2bd883067129c16738b4b29b1991ca30373afba7ff534a7a5b23a9789344b5528f0e7cfecb8c3697754873aadcf93ecdac1eca78d1860f5111a3f80c5af8c40721de75ff77c6e4ddee01bf756f8ba3a55306578712c8c340ab4bd60e9cd9810e46c9f1cfb9466f769a37d5eceb98098ef997718082f9561ebbfc514b02cc07ac750b1a467f58e798f998b46fbc0e05d7069d2c9c8389307e558d472a14d94dbe2,super-glue-pretrained-network:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2058261%2F3414836%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D34f33d0d20f564331a0d247d6fc3e595b5080ab8ba69a7a66abebb501bba278c55adf01c9463e23552ba5d5487268e80a23224ffee01786fb91437396a3b568706e995db54e75620ac14f8207bdccc8d3c8e6b90a957d6dcb2dc338bc4837c67dbaebcf19606b5aa2bdc350953a0effccb830ed0df65b4f9dc8f3d02866c8bc794329a53973890b4c859096d87e91fa4c13fc32b2fc7361741db3cb24f49f45e55a0e15b0f76462108366cdb94edcb7b16b90690b86ed4dc2c0a4730f409e9bd63c97db3128c365e3c9fb080dd475df55b937eccf4cf88e0d398a9b98a3f77c1cab6f54da33b4491f9cfa23a955df1b218165363f244e4fc288ab849482d3e3c,kornia-local-feature-weights:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3117886%2F5373920%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8fa74194a7ac4c0c2893e4f33940f080c9d7e870036c07f3abdb7d2b5256d17aed716071070a1b655e8900fc1fec3d75efb6acc5fbf5260af24c3a6054b1743fbc2509f028f61c89627b50d2f1fb12cffa009c90a647b54bd000aa5b9d8db17282bb76899b11d7d45737851dc197f4f9b37560d78afcfe8d09bcdb1ea03bcdaf9d9406534e5916de7090f0c46562738e19e670f7060c9f7f5d90fa9116e6452ddce9ae88fec975a32ac74fae9f5c4af30947c432abf5b9a3639a2b528da6d8b142582a9b98819d97f30a266f4cd9014a8e2a24e148d853b3dee09b7279e9a72da1c6678cd1e3fdbc59fe9dd95d3873ecd39ecc9167e0b5b7585791f5bd030ad9,imc2024-packages-lightglue-rerun-kornia:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628051%2F7884485%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9fce8669b581cd2efa39d6e216aa7329c9b42e74b9c0f7cd5ce2c7c283d17ab7398bd0c69aecffbd3d5c85ea7d43c3cd67d86536f4a814ece4f4993aff2c6e4a89d72093f5ae8226b30e82fc70968aaa284f06ffd396a4b0beeeee55cf08a30ac1666d8265bee4632d8013905c8d3b09ddaed6a3bed3d20e73798df72d61a90c375f6e5d3e8ba9abe5929df700857f1cab430863d1f71c9e3c6523a27b6141857d6b2a91446bf3c30f768ff776f01d5e253515adacb541be7d280b5627637c2c43faef4b034025a6c3289ecfa4a5b92d894c772f10f29472e15ad1fd27151ec65107e7d915d0f512e0618f1dcc31a1ba0ce373f70b4da2fdce1ddbae75c8f9f1,tf-efficientnet/pytorch/tf-efficientnet-b7/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F2663%2F3736%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dadc1818f4d95ff1f9d92d63411ba60842be4dfba39f201faba40c3af7c82d4c578b293890a42e3d3d21918031c6184fb8af389fd0a18ba0e66583a433c7c3d39ff218a9bfd2721a58c8912d251c9e5f3529c882f08c3e1473956955857ecd28a72f8b4a45f3324f445a0a8e93884bca09ba1455a85ab98c2afa5cb716765b79b15a5079ebc49abb2bb775cc1ac83aace2a68c2c97e58d919c8e75518dd8a2474016d5cadf29dfb6a0d964f903e1bf45538449d843dcebaead99aa0b9af764424a4f03308b7534b5d918e60c908165204b8085351f04dabc8bf42e8e2c62b8cf4da25edcb5826aa232f1623f1ee25f3f5c974c085cbbfce7f6dee79a5e5372905,loftr/pytorch/outdoor/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F2742%2F3840%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1347db2e685149c26e233f4bff4a042eb96f09c5403b0bf4da4b29097a89a35ff0af47fd49fdb34d2cb94a2534e2a2e8c4583499d0d5b97069831a3c6a2d0ac06b9ea5c51a508a8425b0e393fc67ee66b313b8c501061f99afeed071b86efb1278a4a240acb20232dbe47c3dd55fdc138dfea6322a311b1bcca84a35d653e1c10dcd48f3fa35f540eaa3acf7b6a27c8e36d329f2f205143ebddc847f336485b558e131b171fb2608fee53d265a301c16b7d134f2e15fecfa84d9d27bc517bb69d67951f39ae5af70c56293d4d4d47ee01b4762a6a0793b8d0133435093a729c8b952e872fd89599a0e7bfb55e57b5f94efcce5ac0e8cd2fab84104a0792c19d6,disk/pytorch/depth-supervision/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F2747%2F3846%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D31f4f756e9eda9b02f98c00a0b22ffe7ba01fe2e8e6291463f076e75811802adda3801e9130bfeabb9ce367fcf3b9665c3f6fb91e5b3f9a6c17b26a43ef57c9b8d8908c0e9bd617257fb5ca8ddf6cf364a0a4b907219476db1a2192bbbf7e5fd206e32e16ef17e9cbb6abec09cf7299119603dfd22605155f4ebb0ca39ad949cfeba33903c49d771e2fd54a772f5f472dc88688f43d23258af37c799a7470367ae3d1cd591f2a4139779b0758020c74fe29b38c222a9b442f3308f6d3285bf79cd0c64593f923bf02fa5c56721f4f077d655bea3feffdad17a2eb8650498c95e894bbe0d823a67cd283095e11a3932ec4f6ccb5b52ab4ccfd44eaa4f5f0f6f8d,dinov2/pytorch/base/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3326%2F4534%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D859c071a40204b6309293d00a24fa0c4db24e4651ad04b0695dc134d107bf557d2290f5c2776571fc64e9e63144ebd6ebb6d154e5fc5b262cc4afd6e23b826c7b7bffe8829e5a06eb7fa316c22bfedb2175e9c845d62fdaf79301f634591477bca43a433499e15ff109c29115cb5bc365f3f350989f83f218752ab97523a9473439469af57ab84d1e2a2b233fddf88f4387e370501607718c19089d29951f56b3d76df765d157562c49ec122f222109e5188aa3d2d3c87ab7859228e0b0cb8abc9b2f1404833a23ce284336d50cd4c5d41661e711111b85bbf5bd6324a9798e1adbff0d8b3a20a1dd067b0d188985c9cfc31f45601647c753a28e4aff72ea808,lightglue/pytorch/aliked/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14317%2F17191%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070449Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4ec80dec866009b68e4a56c9da14a6622d84c09b866d64f9fdb811117119c59d7331f02ce82ba3cb7a6159e961d89ec58e1aa58c63ceff865913e76539b0cea9fa803a902d9b4be21e66c90d6741faa539d99b03170f1c15ad3e8c6af57471021479eb213a6c024035a0225bb31ff327800bd87993fb9eac18236c9c12a8bc16bb43a618eca34f05612102ae624e675a474eaaa670708faff9ae721f22e274de6cde7d74f4eb537ac328d1b68fab58f7998a3013f4592949d05c9ea4341f351ac6ec160f605f1aae2a73deb21df8558b5b503c1b20dd3ea2a793fa9ddb0350ea6cd9f25dc5bd7333ecef1f1d8b9181bc5e63748b3b8f3ffcf40e8322d571dced,aliked/pytorch/aliked-n16/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14611%2F17555%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T070449Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D54719dccd209bc752f5976bf177199395b413b44bb3f57cff99a46c0473ebc147e0cc6c56618b9e7aa20083f4611c5169833f408fb8940b2d66f6effbb17dcf6d3589d342bd8b90e96009f973b3c0bc080066bac571aa5c217fd3b3b8c2a6be8a3b5fec6b1c42e1c21469e2f4f50df996f3851d9190fce0fcbd6b3fced339a43478cd5f204d33d7df5e12e896d043f8f8badccd98d4689945c1faa1f87a4ae0a593078e416f26826490ae17d6d3a856c19ad14369ad256fc56ba7ad445adeca3f90a17c69cf8530edc4bbea66349aba59090881288d1e275d9a0e5347f9ebf0f5690c2c129811bddbef288a98cb00a15e5e5135950d5e9dd092855a760303db3'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Dependencies

In [ ]:
!python -m pip install --no-deps /kaggle/input/dependencies-imc/pycolmap/pycolmap-0.4.0-cp310-cp310-manylinux2014_x86_64.whl
!python -m pip install --no-deps /kaggle/input/dependencies-imc/safetensors/safetensors-0.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!python -m pip install --no-index --find-links=/kaggle/input/dependencies-imc/transformers/ transformers > /dev/null
!python -m pip install  --no-deps /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl

# dkm
!python -m pip install --no-index --find-links=/kaggle/input/dkm-dependencies/packages einops > /dev/null

Processing /kaggle/input/dependencies-imc/pycolmap/pycolmap-0.4.0-cp310-cp310-manylinux2014_x86_64.whl
Processing /kaggle/input/dependencies-imc/safetensors/safetensors-0.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.4.3
    Uninstalling safetensors-0.4.3:
      Successfully uninstalled safetensors-0.4.3
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl


In [ ]:
# lightglue models
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth
!cp /kaggle/input/pytorch-lightglue-models/* /root/.cache/torch/hub/checkpoints/

# dkm model
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/dkm-dependencies/DKMv3_outdoor.pth /root/.cache/torch/hub/checkpoints/

In [ ]:
%matplotlib inline

In [ ]:
# General utilities
import os
from tqdm import tqdm
from time import time
from fastprogress import progress_bar
import gc
import numpy as np
import pandas as pd
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy

# CV/ML
import cv2
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import kornia as K
import kornia.feature as KF
from PIL import Image
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

import torchvision

# 3D reconstruction
import pycolmap

import glob
import matplotlib
from matplotlib import pyplot as plt

# dkm
import sys
sys.path.append('/kaggle/input/dkm-dependencies/DKM/')
from dkm.utils.utils import tensor_to_pil, get_tuple_transform_ops
from dkm import DKMv3_outdoor

# LoFTR
from kornia.feature import LoFTR

In [ ]:
from lightglue import match_pair
from lightglue import ALIKED, SuperPoint, DoGHardNet, LightGlue
from lightglue.utils import load_image, rbd
from kornia.feature import LoFTR

In [ ]:
print('Kornia version', K.__version__)
print('Pycolmap version', pycolmap.__version__)

Kornia version 0.7.2
Pycolmap version 0.4.0


# Configurations

In [ ]:
class CONFIG:
    use_aliked_lightglue = True
    use_doghardnet_lightglue = False
    use_superpoint_lightglue = False
    use_loftr = False
    use_dkm = False
    use_superglue = False

    params_aliked_lightglue = {
        "num_features" : 8192,
        "detection_threshold" : 0.001,
        "min_matches" : 15,
        "resize_to" : 1024,
    }

    params_doghardnet_lightglue = {
        "num_features" : 8192,
        "detection_threshold" : 0.001,
        "min_matches" : 15,
        "resize_to" : 1024,
    }

    params_superpoint_lightglue = {
        "num_features" : 4096,
        "detection_threshold" : 0.005,
        "min_matches" : 15,
        "resize_to" : 1024,
    }

    params_loftr = {
        "resize_small_edge_to" : 750,
        "min_matches" : 15,
    }

    params_dkm = {
        "num_features" : 2048,
        "detection_threshold" : 0.4,
        "min_matches" : 15,
        "resize_to" : (540, 720),
    }

    params_sg = {
        "sg_config" :
        {
            "superpoint": {
                "nms_radius": 4,
                "keypoint_threshold": 0.0001,
                "max_keypoints": 4096
            },
            "superglue": {
                "weights": "outdoor",
                "sinkhorn_iterations": 10,
                "match_threshold": 0.2,
            },
        },
        "resize_to": 1240,
        "min_matches": 15,
    }

In [ ]:
device=torch.device('cuda')

# COLMAP utilities

In [ ]:
# Code to manipulate a colmap database.
# Forked from https://github.com/colmap/colmap/blob/dev/scripts/python/database.py

# Copyright (c) 2018, ETH Zurich and UNC Chapel Hill.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#
#     * Neither the name of ETH Zurich and UNC Chapel Hill nor the names of
#       its contributors may be used to endorse or promote products derived
#       from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDERS OR CONTRIBUTORS BE
# LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
# CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
# SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
# INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
# CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.
#
# Author: Johannes L. Schoenberger (jsch-at-demuc-dot-de)

# This script is based on an original implementation by True Price.

import sys
import sqlite3
import numpy as np


IS_PYTHON3 = sys.version_info[0] >= 3

MAX_IMAGE_ID = 2**31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    if IS_PYTHON3:
        return array.tostring()
    else:
        return np.getbuffer(array)


def blob_to_array(blob, dtype, shape=(-1,)):
    if IS_PYTHON3:
        return np.fromstring(blob, dtype=dtype).reshape(*shape)
    else:
        return np.frombuffer(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)


    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = \
            lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = \
            lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = \
            lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = \
            lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = \
            lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = \
            lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(self, model, width, height, params,
                   prior_focal_length=False, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (camera_id, model, width, height, array_to_blob(params),
             prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id,
                  prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2],
             prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),))

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),))

    def add_matches(self, image_id1, image_id2, matches):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),))

    def add_two_view_geometry(self, image_id1, image_id2, matches,
                              F=np.eye(3), E=np.eye(3), H=np.eye(3), config=2):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches), config,
             array_to_blob(F), array_to_blob(E), array_to_blob(H)))

# h5 to colmap db

In [ ]:
# Code to interface DISK with Colmap.
# Forked from https://github.com/cvlab-epfl/disk/blob/37f1f7e971cea3055bb5ccfc4cf28bfd643fa339/colmap/h5_to_db.py

#  Copyright [2020] [Michał Tyszkiewicz, Pascal Fua, Eduard Trulls]
#
#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.

import os, argparse, h5py, warnings
import numpy as np
from tqdm import tqdm
from PIL import Image, ExifTags


def get_focal(image_path, err_on_default=False):
    image         = Image.open(image_path)
    max_size      = max(image.size)

    exif = image.getexif()
    focal = None
    if exif is not None:
        focal_35mm = None
        # https://github.com/colmap/colmap/blob/d3a29e203ab69e91eda938d6e56e1c7339d62a99/src/util/bitmap.cc#L299
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_35mm = float(value)
                break

        if focal_35mm is not None:
            focal = focal_35mm / 35. * max_size

    if focal is None:
        if err_on_default:
            raise RuntimeError("Failed to find focal length")

        # failed to find it in exif, use prior
        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size

    return focal

def create_camera(db, image_path, camera_model):
    image         = Image.open(image_path)
    width, height = image.size

    focal = get_focal(image_path)

    if camera_model == 'simple-pinhole':
        model = 0 # simple pinhole
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1 # pinhole
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2 # simple radial
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4 # opencv
        param_arr = np.array([focal, focal, width / 2, height / 2, 0., 0., 0., 0.])

    return db.add_camera(model, width, height, param_arr)


def add_keypoints(db, h5_path, image_path, img_ext, camera_model, single_camera = True):
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')

    camera_id = None
    fname_to_id = {}
    for filename in tqdm(list(keypoint_f.keys())):
        keypoints = keypoint_f[filename][()]

        fname_with_ext = filename# + img_ext
        path = os.path.join(image_path, fname_with_ext)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(fname_with_ext, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id

def add_matches(db, h5_path, fname_to_id):
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')

    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                    continue

                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)

                added.add(pair_id)

                pbar.update(1)

def import_into_colmap(img_dir,
                       feature_dir ='.featureout',
                       database_path = 'colmap.db',
                       img_ext='.jpg'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, img_ext, 'simple-radial', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )

    db.commit()
    return

# Image Pairs

In [ ]:
# We will use ViT global descriptor to get matching shortlists.
def get_global_desc(fnames, model,
                    device =  torch.device('cpu')):
    model = model.eval()
    model= model.to(device)
    config = resolve_data_config({}, model=model)
    transform = create_transform(**config)
    global_descs_convnext=[]
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        img = Image.open(img_fname_full).convert('RGB')
        timg = transform(img).unsqueeze(0).to(device)
        with torch.no_grad(), torch.cuda.amp.autocast():
            desc = model.forward_features(timg.to(device)).mean(dim=(-1,2))#
            #print (desc.shape)
            desc = desc.view(1, -1)
            desc_norm = F.normalize(desc, dim=1, p=2)
        #print (desc_norm)
        global_descs_convnext.append(desc_norm.detach().cpu())
    global_descs_all = torch.cat(global_descs_convnext, dim=0)
    return global_descs_all.to(torch.float32)

def convert_1d_to_2d(idx, num_images):
    idx1 = idx // num_images
    idx2 = idx % num_images
    return (idx1, idx2)

def get_pairs_from_distancematrix(mat):
    pairs = [ convert_1d_to_2d(idx, mat.shape[0]) for idx in np.argsort(mat.flatten())]
    pairs = [ pair for pair in pairs if pair[0] < pair[1] ]
    return pairs

def get_img_pairs_exhaustive(img_fnames, model, device):
    #index_pairs = []
    #for i in range(len(img_fnames)):
    #    for j in range(i+1, len(img_fnames)):
    #        index_pairs.append((i,j))
    #return index_pairs
    descs = get_global_desc(img_fnames, model, device=device)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()
    matching_list = get_pairs_from_distancematrix(dm)
    return matching_list


def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6, # should be strict
                              min_pairs = 20,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)

    model = timm.create_model('tf_efficientnet_b7',
                              checkpoint_path='/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b7/1/tf_efficientnet_b7_ra-6c08e654.pth')
    model.eval()
    descs = get_global_desc(fnames, model, device=device)

    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames, model, device)

    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()
    # removing half
    mask = dm <= sim_th
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < 1000:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

# Keypoints: LightGlue series

In [ ]:
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

def detect_common(img_fnames,
                  model_name,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 1024,
                  detection_threshold = 0.01,
                  device=torch.device('cpu')):

    dict_model = {
        "aliked" : ALIKED,
        "superpoint" : SuperPoint,
        "doghardnet" : DoGHardNet,
    }
    extractor_class = dict_model[model_name]

    dtype = torch.float32 # ALIKED has issues with float16
    extractor = extractor_class(
        max_num_keypoints=num_features, detection_threshold=detection_threshold, resize=resize_to
    ).eval().to(device, dtype)
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    with h5py.File(f'{feature_dir}/keypoints_{model_name}.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors_{model_name}.h5', mode='w') as f_desc:
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = extractor.extract(image0)  # auto-resize the image, disable with resize=None
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
                print(f"{model_name} > kpts.shape={kpts.shape}, descs.shape={descs.shape}")
    return

def match_with_lightglue_common(img_fnames, model_name,
                   index_pairs, file_keypoints,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=15,verbose=True):
    lg_matcher = KF.LightGlueMatcher(model_name, {"width_confidence": -1,
                                                "depth_confidence": -1,
                                                 "mp": True if 'cuda' in str(device) else False}).eval().to(device)

    cnt_pairs = 0
    with h5py.File(f'{feature_dir}/keypoints_{model_name}.h5', mode='r') as f_kp, \
        h5py.File(f'{feature_dir}/descriptors_{model_name}.h5', mode='r') as f_desc, \
        h5py.File(file_keypoints, mode='w') as f_match:
        for pair_idx in tqdm(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]

            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                         desc2,
                                         KF.laf_from_center_scale_ori(kp1[None]),
                                         KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0:
                continue
            n_matches = len(idxs)
            kp1 = kp1[idxs[:,0], :].cpu().numpy().reshape(-1, 2).astype(np.float32)
            kp2 = kp2[idxs[:,1], :].cpu().numpy().reshape(-1, 2).astype(np.float32)
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                group.create_dataset(key2, data=np.concatenate([kp1, kp2], axis=1))
                cnt_pairs+=1
                print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair({model_name}+lightglue)')
            else:
                print (f'{key1}-{key2}: {n_matches} matches --> skipped')
    return

def detect_lightglue_common(
    img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints,
    resize_to=1024,
    detection_threshold=0.01,
    num_features=4096,
    min_matches=15,
):
    t=time()
    detect_common(
        img_fnames, model_name, feature_dir,
        resize_to=resize_to,
        num_features=num_features,
        detection_threshold=detection_threshold,
        device=device
    )
    gc.collect()
    match_with_lightglue_common(
        img_fnames, model_name, index_pairs, file_keypoints,
        feature_dir=feature_dir,
        min_matches=min_matches,
        device=device
    )
    t=time() -t
    print(f'Features matched in  {t:.4f} sec ({model_name}+LightGlue)')
    return t

# Keypoints: SuperGlue

In [ ]:
import sys
sys.path.append("../input/super-glue-pretrained-network")
from models.matching import Matching
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          process_resize, frame2tensor,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

from torch.nn import functional as torchF  # For resizing tensor

def sg_imread(path):
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    return image

# Preprocess
def sg_read_image(image, device, resize):
    w, h = image.shape[1], image.shape[0]
    w_new, h_new = process_resize(w, h, [resize,])

    unit_shape = 8
    w_new = w_new // unit_shape * unit_shape
    h_new = h_new // unit_shape * unit_shape

    scales = (float(w) / float(w_new), float(h) / float(h_new))
    image = cv2.resize(image.astype('float32'), (w_new, h_new))

    inp = frame2tensor(image, "cpu")
    return image, inp, scales, (h, w)

class SGDataset(Dataset):
    def __init__(self, fnames1, fnames2, resize_to, device):
        self.fnames1 = fnames1
        self.fnames2 = fnames2
        self.resize_to = resize_to
        self.device = device

    def __len__(self):
        return len(self.fnames1)

    def __getitem__(self, idx):
        fname1 = self.fnames1[idx]
        fname2 = self.fnames2[idx]

        im1, im2 = cv2.imread(fname1, cv2.IMREAD_GRAYSCALE), cv2.imread(fname2, cv2.IMREAD_GRAYSCALE)
        _, image1, scale1, ori_shape1 = sg_read_image(im1, self.device, self.resize_to)
        _, image2, scale2, ori_shape2 = sg_read_image(im2, self.device, self.resize_to)
        return image1, image2, torch.tensor([idx]), torch.tensor(ori_shape1), torch.tensor(ori_shape2)

def get_superglue_dataloader(images1, images2, resize_to, device, batch_size=1):
    dataset = SGDataset(images1, images2, resize_to, device)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    return dataloader

def detect_superglue(
    img_fnames, index_pairs, feature_dir, device, sg_config, file_keypoints,
    resize_to=750, min_matches=15
):
    t=time()

    matcher_superglue = Matching(sg_config).eval().to(device)

    fnames1, fnames2, idxs1, idxs2 = [], [], [], []
    for pair_idx in progress_bar(index_pairs):
        idx1, idx2 = pair_idx
        fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
        fnames1.append(fname1)
        fnames2.append(fname2)
        idxs1.append(idx1)
        idxs2.append(idx2)

    dataloader = get_superglue_dataloader( fnames1, fnames2, resize_to, device)

    cnt_pairs = 0

    with h5py.File(file_keypoints, mode='w') as f_match:
        for X in dataloader:
            image1, image2, idx, ori_shape_1, ori_shape_2 = X

            fname1, fname2 = fnames1[idx], fnames2[idx]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]

            pred = matcher_superglue({"image0": image1[0].to(device), "image1": image2[0].to(device)})
            pred = {k: v[0].detach().cpu().numpy().copy() for k, v in pred.items()}
            mkpts1, mkpts2 = pred["keypoints0"], pred["keypoints1"]
            matches, conf = pred["matches0"], pred["matching_scores0"]

            valid = matches > -1
            mkpts1 = mkpts1[valid]
            mkpts2 = mkpts2[matches[valid]]
            mconf = conf[valid]

            ori_shape_1 = ori_shape_1[0].numpy()
            ori_shape_2 = ori_shape_2[0].numpy()

            # Scaling coords
            mkpts1[:,0] = mkpts1[:,0] * ori_shape_1[1] / image1[0].shape[3]   # X
            mkpts1[:,1] = mkpts1[:,1] * ori_shape_1[0] / image1[0].shape[2]   # Y
            mkpts2[:,0] = mkpts2[:,0] * ori_shape_2[1] / image2[0].shape[3]   # X
            mkpts2[:,1] = mkpts2[:,1] * ori_shape_2[0] / image2[0].shape[2]   # Y

            n_matches = mconf.shape[0]

            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                group.create_dataset(key2, data=np.concatenate([mkpts1, mkpts2], axis=1).astype(np.float32))
                cnt_pairs+=1
                print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair(superglue)')
            else:
                print (f'{key1}-{key2}: {n_matches} matches --> skipped')

    gc.collect()
    t=time() -t
    print(f'Features matched in  {t:.4f} sec')
    return t

# Keypoints: DKM

In [ ]:
class DKMDataset(Dataset):
    def __init__(self, fnames1, fnames2, resize_to, device):
        self.fnames1 = fnames1
        self.fnames2 = fnames2
        self.resize_to = resize_to
        self.device = device
        self.test_transform = get_tuple_transform_ops(
            resize=self.resize_to, normalize=True
        )


    def __len__(self):
        return len(self.fnames1)

    def __getitem__(self, idx):
        fname1 = self.fnames1[idx]
        fname2 = self.fnames2[idx]

        im1, im2 = Image.open(fname1), Image.open(fname2)
        ori_shape_1 = im1.size
        ori_shape_2 = im2.size
        image1, image2 = self.test_transform((im1, im2))
        return image1, image2, torch.tensor([idx]), torch.tensor(ori_shape_1), torch.tensor(ori_shape_2)

def get_dkm_dataloader(images1, images2, resize_to, device, batch_size=4):
    dataset = DKMDataset(images1, images2, resize_to, device)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    return dataloader

def get_dkm_mkpts(dkm_model, bimgs1, bimgs2, shapes1, shapes2, detection_threshold=0.5, num_features = 2000, min_matches=15):
    dense_matches, dense_certainty = dkm_model.match(bimgs1, bimgs2, batched=True)
    print("***", dense_matches.shape, dense_certainty.shape)

    store_mkpts1, store_mkpts2, store_mconf = [], [], []
    # drop low confidence pairs
    for b in range(dense_matches.shape[0]):
        u_dense_matches = dense_matches[b, dense_certainty[b,...].sqrt() >= detection_threshold, :]
        u_dense_certainty = dense_certainty[b, dense_certainty[b,...].sqrt() >= detection_threshold]

        if u_dense_matches.shape[0] > num_features:
            u_dense_matches, u_dense_certainty = dkm_model.sample( u_dense_matches, u_dense_certainty, num=num_features)

        u_dense_matches = u_dense_matches.reshape((-1, 4))
        u_dense_certainty = u_dense_certainty.reshape((-1,))

        mkpts1 = u_dense_matches[:, :2]
        mkpts2 = u_dense_matches[:, 2:]

        w1, h1 = shapes1[b, :]
        w2, h2 = shapes2[b, :]

        mkpts1[:, 0] = ((mkpts1[:, 0] + 1)/2) * w1
        mkpts1[:, 1] = ((mkpts1[:, 1] + 1)/2) * h1

        mkpts2[:, 0] = ((mkpts2[:, 0] + 1)/2) * w2
        mkpts2[:, 1] = ((mkpts2[:, 1] + 1)/2) * h2

        mkpts1 = mkpts1.cpu().detach().numpy()
        mkpts2 = mkpts2.cpu().detach().numpy()
        mconf  = u_dense_certainty.sqrt().cpu().detach().numpy()


        if mconf.shape[0] > min_matches:
            try:
                # calc Fundamental matrix from keypoints
                F, inliers = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, 0.200, 0.999, 2000)
                inliers = inliers > 0
                mkpts1 = mkpts1[inliers[:,0]]
                mkpts2 = mkpts2[inliers[:,0]]
                mconf  = mconf[inliers[:,0]]
                #print("---", mconf.shape)
                if mconf.shape[0] > 3000:
                    rand_idx = np.random.choice(range(mconf.shape[0]), 3000, replace=False)
                    mkpts1 = mkpts1[rand_idx, :]
                    mkpts2 = mkpts2[rand_idx, :]
                    mconf  = mconf[rand_idx]
            except:
                mkpts1 = np.empty((0,2))
                mkpts2 = np.empty((0,2))
                mconf = np.empty((0,))

        store_mkpts1.append(mkpts1)
        store_mkpts2.append(mkpts2)
        store_mconf.append(mconf)
    return store_mkpts1, store_mkpts2, store_mconf

def detect_dkm(
    img_fnames, index_pairs, feature_dir, device,
    resize_to=(540, 720),
    detection_threshold=0.4,
    num_features=2000,
    min_matches=15,
):
    t=time()
    dkm_model = DKMv3_outdoor(device=device)
    dkm_model.upsample_preds=False

    fnames1, fnames2 = [], []
    for pair_idx in progress_bar(index_pairs):
        idx1, idx2 = pair_idx
        fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
        fnames1.append(fname1)
        fnames2.append(fname2)

    cnt_pairs = 0
    with h5py.File(f'{feature_dir}/matches_dkm.h5', mode='w') as f_match:
        dataloader = get_dkm_dataloader(fnames1, fnames2, resize_to, device, batch_size=4)
        for X in tqdm(dataloader):
            images1, images2, idxs, shapes1, shapes2 = X
            store_mkpts1, store_mkpts2, store_mconf = get_dkm_mkpts(
                dkm_model, images1.to(device), images2.to(device), shapes1, shapes2,
                detection_threshold=detection_threshold, num_features = num_features, min_matches=min_matches,
            )

            for b in range(images1.shape[0]):
                mkpts1 = store_mkpts1[b]
                mkpts2 = store_mkpts2[b]
                mconf = store_mconf[b]
                file1 = fnames1[idxs[b]]
                file2 = fnames2[idxs[b]]
                key1, key2 = file1.split('/')[-1], file2.split('/')[-1]

                n_matches = mconf.shape[0]
                print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair(dkm)')

                group  = f_match.require_group(key1)
                if n_matches >= min_matches:
                    group.create_dataset(key2, data=np.concatenate([mkpts1, mkpts2], axis=1).astype(np.float32))
                    cnt_pairs+=1
    gc.collect()
    t=time() -t
    print(f'Features matched in  {t:.4f} sec')
    return t

# Keypoints: LoFTR

In [ ]:
class LoFTRDataset(Dataset):
    def __init__(self, fnames1, fnames2, idxs1, idxs2, resize_small_edge_to, device):
        self.fnames1 = fnames1
        self.fnames2 = fnames2
        self.keys1 = [ fname.split('/')[-1] for fname in fnames1 ]
        self.keys2 = [ fname.split('/')[-1] for fname in fnames2 ]
        self.idxs1 = idxs1
        self.idxs2 = idxs2
        self.resize_small_edge_to = resize_small_edge_to
        self.device = device
        self.round_unit = 16

    def __len__(self):
        return len(self.images1)

    def load_torch_image(self, fname, device):
        img = cv2.imread(fname)
        original_shape = img.shape
        ratio = self.resize_small_edge_to / min([img.shape[0], img.shape[1]])
        w = int(img.shape[1] * ratio) # int( (img.shape[1] * ratio) // self.round_unit * self.round_unit )
        h = int(img.shape[0] * ratio) # int( (img.shape[0] * ratio) // self.round_unit * self.round_unit )
        img_resized = cv2.resize(img, (w, h))
        img_resized = K.image_to_tensor(img_resized, False).float() /255.
        img_resized = K.color.bgr_to_rgb(img_resized)
        img_resized = K.color.rgb_to_grayscale(img_resized)
        return img_resized.to(device), original_shape

    def __getitem__(self, idx):
        fname1 = self.fnames1[idx]
        fname2 = self.fnames2[idx]
        image1, ori_shape_1 = self.load_torch_image(fname1, device)
        image2, ori_shape_2 = self.load_torch_image(fname2, device)

        return image1, image2, self.keys1[idx], self.keys2[idx], self.idxs1[idx], self.idxs2[idx], ori_shape_1, ori_shape_2

def get_loftr_dataloader(images1, images2, idxs1, idxs2, resize_small_edge_to, device, batch_size=1):
    dataset = LoFTRDataset(images1, images2, idxs1, idxs2, resize_small_edge_to, device)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    return dataset

def detect_loftr(img_fnames, index_pairs, feature_dir, device, file_keypoints, resize_small_edge_to=750, min_matches=15):
    t=time()

    matcher = LoFTR(pretrained=None)
    matcher.load_state_dict(torch.load("../input/loftr/pytorch/outdoor/1/loftr_outdoor.ckpt")['state_dict'])
    matcher = matcher.to(device).eval()

    fnames1, fnames2, idxs1, idxs2 = [], [], [], []
    for pair_idx in progress_bar(index_pairs):
        idx1, idx2 = pair_idx
        fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
        fnames1.append(fname1)
        fnames2.append(fname2)
        idxs1.append(idx1)
        idxs2.append(idx2)


    dataloader = get_loftr_dataloader( fnames1, fnames2, idxs1, idxs2, resize_small_edge_to, device)

    cnt_pairs = 0

    with h5py.File(file_keypoints, mode='w') as f_match:
        store_mkpts = {}
        for X in tqdm(dataloader):
            image1, image2, key1, key2, idx1, idx2, ori_shape_1, ori_shape_2 = X
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]

            with torch.no_grad():
                correspondences = matcher( {"image0": image1.to(device),"image1": image2.to(device)} )
                mkpts1 = correspondences['keypoints0'].cpu().numpy()
                mkpts2 = correspondences['keypoints1'].cpu().numpy()
                mconf  = correspondences['confidence'].cpu().numpy()

            mkpts1[:,0] *= (float(ori_shape_1[1]) / float(image1.shape[3]))
            mkpts1[:,1] *= (float(ori_shape_1[0]) / float(image1.shape[2]))

            mkpts2[:,0] *= (float(ori_shape_2[1]) / float(image2.shape[3]))
            mkpts2[:,1] *= (float(ori_shape_2[0]) / float(image2.shape[2]))

            n_matches = mconf.shape[0]

            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                group.create_dataset(key2, data=np.concatenate([mkpts1, mkpts2], axis=1).astype(np.float32))
                cnt_pairs+=1
                print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair(loftr)')
            else:
                print (f'{key1}-{key2}: {n_matches} matches --> skipped')
    gc.collect()
    t=time() -t
    print(f'Features matched in  {t:.4f} sec')
    return t

# Keypoints: DKM

In [ ]:
class DKMDataset(Dataset):
    def __init__(self, fnames1, fnames2, resize_to, device):
        self.fnames1 = fnames1
        self.fnames2 = fnames2
        self.resize_to = resize_to
        self.device = device
        self.test_transform = get_tuple_transform_ops(
            resize=self.resize_to, normalize=True
        )


    def __len__(self):
        return len(self.fnames1)

    def __getitem__(self, idx):
        fname1 = self.fnames1[idx]
        fname2 = self.fnames2[idx]

        im1, im2 = Image.open(fname1), Image.open(fname2)
        ori_shape_1 = im1.size
        ori_shape_2 = im2.size
        image1, image2 = self.test_transform((im1, im2))
        return image1, image2, torch.tensor([idx]), torch.tensor(ori_shape_1), torch.tensor(ori_shape_2)

def get_dkm_dataloader(images1, images2, resize_to, device, batch_size=4):
    dataset = DKMDataset(images1, images2, resize_to, device)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    return dataloader

def get_dkm_mkpts(dkm_model, bimgs1, bimgs2, shapes1, shapes2, detection_threshold=0.5, num_features = 2000, min_matches=15):
    dense_matches, dense_certainty = dkm_model.match(bimgs1, bimgs2, batched=True)

    store_mkpts1, store_mkpts2, store_mconf = [], [], []
    # drop low confidence pairs
    for b in range(dense_matches.shape[0]):
        u_dense_matches = dense_matches[b, dense_certainty[b,...].sqrt() >= detection_threshold, :]
        u_dense_certainty = dense_certainty[b, dense_certainty[b,...].sqrt() >= detection_threshold]

        if u_dense_matches.shape[0] > num_features:
            u_dense_matches, u_dense_certainty = dkm_model.sample( u_dense_matches, u_dense_certainty, num=num_features)

        u_dense_matches = u_dense_matches.reshape((-1, 4))
        u_dense_certainty = u_dense_certainty.reshape((-1,))

        mkpts1 = u_dense_matches[:, :2]
        mkpts2 = u_dense_matches[:, 2:]

        w1, h1 = shapes1[b, :]
        w2, h2 = shapes2[b, :]

        mkpts1[:, 0] = ((mkpts1[:, 0] + 1)/2) * w1
        mkpts1[:, 1] = ((mkpts1[:, 1] + 1)/2) * h1

        mkpts2[:, 0] = ((mkpts2[:, 0] + 1)/2) * w2
        mkpts2[:, 1] = ((mkpts2[:, 1] + 1)/2) * h2

        mkpts1 = mkpts1.cpu().detach().numpy()
        mkpts2 = mkpts2.cpu().detach().numpy()
        mconf  = u_dense_certainty.sqrt().cpu().detach().numpy()

        if mconf.shape[0] > min_matches:
            try:
                # calc Fundamental matrix from keypoints
                F, inliers = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, 0.200, 0.999, 2000)
                inliers = inliers > 0
                mkpts1 = mkpts1[inliers[:,0]]
                mkpts2 = mkpts2[inliers[:,0]]
                mconf  = mconf[inliers[:,0]]
            except:
                pass
        store_mkpts1.append(mkpts1)
        store_mkpts2.append(mkpts2)
        store_mconf.append(mconf)
    return store_mkpts1, store_mkpts2, store_mconf

def detect_dkm(
    img_fnames, index_pairs, feature_dir, device, file_keypoints,
    resize_to=(540, 720),
    detection_threshold=0.4,
    num_features=2000,
    min_matches=15
):
    t=time()
    dkm_model = DKMv3_outdoor(device=device)
    dkm_model.upsample_preds=False

    fnames1, fnames2 = [], []
    for pair_idx in progress_bar(index_pairs):
        idx1, idx2 = pair_idx
        fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
        fnames1.append(fname1)
        fnames2.append(fname2)

    cnt_pairs = 0
    with h5py.File(file_keypoints, mode='w') as f_match:
        dataloader = get_dkm_dataloader(fnames1, fnames2, resize_to, device, batch_size=4)
        for X in tqdm(dataloader):
            images1, images2, idxs, shapes1, shapes2 = X
            store_mkpts1, store_mkpts2, store_mconf = get_dkm_mkpts(
                dkm_model, images1.to(device), images2.to(device), shapes1, shapes2,
                detection_threshold=detection_threshold, num_features = num_features, min_matches=min_matches,
            )

            for b in range(images1.shape[0]):
                mkpts1 = store_mkpts1[b]
                mkpts2 = store_mkpts2[b]
                mconf = store_mconf[b]
                file1 = fnames1[idxs[b]]
                file2 = fnames2[idxs[b]]
                key1, key2 = file1.split('/')[-1], file2.split('/')[-1]

                n_matches = mconf.shape[0]

                group  = f_match.require_group(key1)
                if n_matches >= min_matches:
                    group.create_dataset(key2, data=np.concatenate([mkpts1, mkpts2], axis=1).astype(np.float32))
                    cnt_pairs+=1
                    print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair(dkm)')
                else:
                    print (f'{key1}-{key2}: {n_matches} matches --> skipped')

    gc.collect()
    t=time() -t
    print(f'Features matched in  {t:.4f} sec')
    return t

# Keypoints merger

In [ ]:
def get_unique_idxs(A, dim=0):
    # https://stackoverflow.com/questions/72001505/how-to-get-unique-elements-and-their-firstly-appeared-indices-of-a-pytorch-tenso
    unique, idx, counts = torch.unique(A, dim=dim, sorted=True, return_inverse=True, return_counts=True)
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0],device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    return first_indices

def get_keypoint_from_h5(fp, key1, key2):
    rc = -1
    try:
        kpts = np.array(fp[key1][key2])
        rc = 0
        return (rc, kpts)
    except:
        return (rc, None)

def get_keypoint_from_multi_h5(fps, key1, key2):
    list_mkpts = []
    for fp in fps:
        rc, mkpts = get_keypoint_from_h5(fp, key1, key2)
        if rc == 0:
            list_mkpts.append(mkpts)
    if len(list_mkpts) > 0:
        list_mkpts = np.concatenate(list_mkpts, axis=0)
    else:
        list_mkpts = None
    return list_mkpts

def keypoints_merger(
    img_fnames,
    index_pairs,
    files_keypoints,
    feature_dir = 'featureout',
):
    # open h5 files
    fps = [ h5py.File(file, mode="r") for file in files_keypoints ]

    # temprary file
    with h5py.File(f'{feature_dir}/merge_tmp.h5', mode='w') as f_match:
        counter = 0
        for pair_idx in progress_bar(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]

            # extract keypoints
            mkpts = get_keypoint_from_multi_h5(fps, key1, key2)
            if mkpts is None:
                print(f"skipped key1={key1}, key2={key2}")
                continue

            print (f'{key1}-{key2}: {mkpts.shape[0]} matches')
            # regist tmp file
            group  = f_match.require_group(key1)
            group.create_dataset(key2, data=mkpts)
            counter += 1
    print( f"Ensembled pairs : {counter} pairs" )
    for fp in fps:
        fp.close()

    # Let's find unique loftr pixels and group them together.
    kpts = defaultdict(list)
    match_indexes = defaultdict(dict)
    total_kpts=defaultdict(int)
    with h5py.File(f'{feature_dir}/merge_tmp.h5', mode='r') as f_match:
        for k1 in f_match.keys():
            group  = f_match[k1]
            for k2 in group.keys():
                matches = group[k2][...]
                total_kpts[k1]
                kpts[k1].append(matches[:, :2])
                kpts[k2].append(matches[:, 2:])
                current_match = torch.arange(len(matches)).reshape(-1, 1).repeat(1, 2)
                current_match[:, 0]+=total_kpts[k1]
                current_match[:, 1]+=total_kpts[k2]
                total_kpts[k1]+=len(matches)
                total_kpts[k2]+=len(matches)
                match_indexes[k1][k2]=current_match

    for k in kpts.keys():
        kpts[k] = np.round(np.concatenate(kpts[k], axis=0))
    unique_kpts = {}
    unique_match_idxs = {}
    out_match = defaultdict(dict)
    for k in kpts.keys():
        uniq_kps, uniq_reverse_idxs = torch.unique(torch.from_numpy(kpts[k]),dim=0, return_inverse=True)
        unique_match_idxs[k] = uniq_reverse_idxs
        unique_kpts[k] = uniq_kps.numpy()
    for k1, group in match_indexes.items():
        for k2, m in group.items():
            m2 = deepcopy(m)
            m2[:,0] = unique_match_idxs[k1][m2[:,0]]
            m2[:,1] = unique_match_idxs[k2][m2[:,1]]
            mkpts = np.concatenate([unique_kpts[k1][ m2[:,0]],
                                    unique_kpts[k2][  m2[:,1]],
                                   ],
                                   axis=1)
            unique_idxs_current = get_unique_idxs(torch.from_numpy(mkpts), dim=0)
            m2_semiclean = m2[unique_idxs_current]
            unique_idxs_current1 = get_unique_idxs(m2_semiclean[:, 0], dim=0)
            m2_semiclean = m2_semiclean[unique_idxs_current1]
            unique_idxs_current2 = get_unique_idxs(m2_semiclean[:, 1], dim=0)
            m2_semiclean2 = m2_semiclean[unique_idxs_current2]
            out_match[k1][k2] = m2_semiclean2.numpy()
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp:
        for k, kpts1 in unique_kpts.items():
            f_kp[k] = kpts1

    with h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for k1, gr in out_match.items():
            group  = f_match.require_group(k1)
            for k2, match in gr.items():
                group[k2] = match
    return

# Submission utilities

In [ ]:
def arr_to_str(a):
    return ';'.join([str(x) for x in a.reshape(-1)])

# Function to create a submission file.
def create_submission(out_results, data_dict):
    with open(f'submission.csv', 'w') as f:
        f.write('image_path,dataset,scene,rotation_matrix,translation_vector\n')
        for dataset in data_dict:
            if dataset in out_results:
                res = out_results[dataset]
            else:
                res = {}
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print (image)
                        R = scene_res[image]['R'].reshape(-1)
                        T = scene_res[image]['t'].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    f.write(f'{image},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n')

# Main

In [ ]:
src = '/kaggle/input/image-matching-challenge-2024'
DEBUG = False
DUMP = False

# Get data from csv.
data_dict = {}
with open(f'{src}/sample_submission.csv', 'r') as f:
    for i, l in enumerate(f):
        # Skip header.
        if l and i > 0:
            image, dataset, scene, _, _ = l.strip().split(',')
            if dataset not in data_dict:
                data_dict[dataset] = {}
            if scene not in data_dict[dataset]:
                data_dict[dataset][scene] = []
            data_dict[dataset][scene].append(image)

            #if len(data_dict[dataset][scene]) == 21:
            #    break

In [ ]:
for dataset in data_dict:
    for scene in data_dict[dataset]:
        print(f'{dataset} / {scene} -> {len(data_dict[dataset][scene])} images')

church / church -> 41 images


In [ ]:
out_results = {}
timings = {"shortlisting":[],
           "feature_detection": [],
           "feature_matching":[],
           "RANSAC": [],
           "Reconstruction": []}

In [ ]:
gc.collect()
datasets = []
for dataset in data_dict:
    datasets.append(dataset)

for dataset in datasets:
    print(dataset)
    if dataset not in out_results:
        out_results[dataset] = {}
    for scene in data_dict[dataset]:
        print(scene)
        # Fail gently if the notebook has not been submitted and the test data is not populated.
        # You may want to run this on the training data in that case?
        img_dir = f'{src}/test/{dataset}/images'
        if not os.path.exists(img_dir):
            continue
        # Wrap the meaty part in a try-except block.
        try:
            out_results[dataset][scene] = {}
            img_fnames = [f'{src}/{x}' for x in data_dict[dataset][scene]]
            print (f"Got {len(img_fnames)} images")
            feature_dir = f'featureout/{dataset}_{scene}'
            if not os.path.isdir(feature_dir):
                os.makedirs(feature_dir, exist_ok=True)

            #############################################################
            # get image pairs
            #############################################################
            t=time()
            index_pairs = get_image_pairs_shortlist(img_fnames,
                                  sim_th = 1.0, # should be strict
                                  min_pairs = 50, # we select at least min_pairs PER IMAGE with biggest similarity
                                  exhaustive_if_less = 50,
                                  device=device)
            t=time() -t
            timings['shortlisting'].append(t)
            print (f'{len(index_pairs)}, pairs to match, {t:.4f} sec')
            gc.collect()

            #############################################################
            # get keypoints
            #############################################################
            files_keypoints = []
            if CONFIG.use_superglue:
                resize_to = CONFIG.params_sg["resize_to"]
                file_keypoints = f"{feature_dir}/matches_superglue_{resize_to}pix.h5"
                !rm -rf {file_keypoints}
                t = detect_superglue(
                    img_fnames, index_pairs, feature_dir, device,
                    CONFIG.params_sg["sg_config"], file_keypoints,
                    resize_to=CONFIG.params_sg["resize_to"],
                    min_matches=CONFIG.params_sg["min_matches"],
                )
                gc.collect()
                files_keypoints.append( file_keypoints )
                timings['feature_matching'].append(t)

            if CONFIG.use_aliked_lightglue:
                model_name = "aliked"
                file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
                t = detect_lightglue_common(
                    img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints,
                    resize_to=CONFIG.params_aliked_lightglue["resize_to"],
                    detection_threshold=CONFIG.params_aliked_lightglue["detection_threshold"],
                    num_features=CONFIG.params_aliked_lightglue["num_features"],
                    min_matches=CONFIG.params_aliked_lightglue["min_matches"],
                )
                gc.collect()
                files_keypoints.append(file_keypoints)
                timings['feature_matching'].append(t)

            if CONFIG.use_doghardnet_lightglue:
                model_name = "doghardnet"
                file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
                t = detect_lightglue_common(
                    img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints,
                    resize_to=CONFIG.params_doghardnet_lightglue["resize_to"],
                    detection_threshold=CONFIG.params_doghardnet_lightglue["detection_threshold"],
                    num_features=CONFIG.params_doghardnet_lightglue["num_features"],
                    min_matches=CONFIG.params_doghardnet_lightglue["min_matches"],
                )
                gc.collect()
                files_keypoints.append(file_keypoints)
                timings['feature_matching'].append(t)

            if CONFIG.use_superpoint_lightglue:
                model_name = "superpoint"
                file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
                t = detect_lightglue_common(
                    img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints,
                    resize_to=CONFIG.params_superpoint_lightglue["resize_to"],
                    detection_threshold=CONFIG.params_superpoint_lightglue["detection_threshold"],
                    num_features=CONFIG.params_superpoint_lightglue["num_features"],
                    min_matches=CONFIG.params_superpoint_lightglue["min_matches"],
                )
                gc.collect()
                files_keypoints.append(file_keypoints)
                timings['feature_matching'].append(t)

            if CONFIG.use_loftr:
                file_keypoints = f'{feature_dir}/matches_loftr_{CONFIG.params_loftr["resize_small_edge_to"]}pix.h5'
                t = detect_loftr(
                    img_fnames, index_pairs, feature_dir, device, file_keypoints,
                    resize_small_edge_to=CONFIG.params_loftr["resize_small_edge_to"],
                    min_matches=CONFIG.params_loftr["min_matches"],
                )
                gc.collect()
                files_keypoints.append( file_keypoints )
                timings['feature_matching'].append(t)

            if CONFIG.use_dkm:
                file_keypoints = f'{feature_dir}/matches_dkm.h5'
                t = detect_dkm(
                    img_fnames, index_pairs, feature_dir, device, file_keypoints,
                    resize_to=CONFIG.params_dkm["resize_to"],
                    detection_threshold=CONFIG.params_dkm["detection_threshold"],
                    num_features=CONFIG.params_dkm["num_features"],
                    min_matches=CONFIG.params_dkm["min_matches"]
                )
                gc.collect()
                files_keypoints.append(file_keypoints)
                timings['feature_matching'].append(t)

            #############################################################
            # merge keypoints
            #############################################################
            keypoints_merger(
                img_fnames,
                index_pairs,
                files_keypoints,
                feature_dir = feature_dir,
            )

            #############################################################
            # regist keypoints from h5 into colmap db
            #############################################################
            database_path = f'{feature_dir}/colmap.db'
            if os.path.isfile(database_path):
                os.remove(database_path)
            gc.collect()
            import_into_colmap(img_dir, feature_dir=feature_dir,database_path=database_path)
            output_path = f'{feature_dir}/colmap_rec'

            #############################################################
            # Calculate fundamental matrix with colmap api
            #############################################################
            t=time()
            options = pycolmap.SiftMatchingOptions()
            options.confidence = 0.9999
            options.max_num_trials = 20000
            pycolmap.match_exhaustive(database_path, sift_options=options)
            t=time() - t
            timings['RANSAC'].append(t)
            print(f'RANSAC in  {t:.4f} sec')

            #############################################################
            # Execute bundle adjustmnet with colmap api
            # --> Bundle adjustment Calcs Camera matrix, R and t
            #############################################################
            t=time()
            # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
            mapper_options = pycolmap.IncrementalMapperOptions()
            mapper_options.min_model_size = 3
            os.makedirs(output_path, exist_ok=True)
            maps = pycolmap.incremental_mapping(database_path=database_path, image_path=img_dir, output_path=output_path, options=mapper_options)
            print(maps)
            clear_output(wait=False)
            t=time() - t
            timings['Reconstruction'].append(t)
            print(f'Reconstruction done in  {t:.4f} sec')

            #############################################################
            # Extract R,t from maps
            #############################################################
            imgs_registered  = 0
            best_idx = None
            list_num_images = []
            print ("Looking for the best reconstruction")
            if isinstance(maps, dict):
                for idx1, rec in maps.items():
                    print (idx1, rec.summary())
                    list_num_images.append( len(rec.images) )
                    if len(rec.images) > imgs_registered:
                        imgs_registered = len(rec.images)
                        best_idx = idx1
            list_num_images = np.array(list_num_images)
            print(f"list_num_images = {list_num_images}")
            if best_idx is not None:
                print (maps[best_idx].summary())
                for k, im in maps[best_idx].images.items():
                    key1 = f'test/{dataset}/images/{im.name}'
                    out_results[dataset][scene][key1] = {}
                    out_results[dataset][scene][key1]["R"] = deepcopy(im.rotmat())
                    out_results[dataset][scene][key1]["t"] = deepcopy(np.array(im.tvec))

            print(f'Registered: {dataset} / {scene} -> {len(out_results[dataset][scene])} images')
            print(f'Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images')
            create_submission(out_results, data_dict)
            gc.collect()
        except:
            pass

Reconstruction done in  318.2972 sec
Looking for the best reconstruction
0 Reconstruction:
	num_reg_images = 39
	num_cameras = 39
	num_points3D = 22272
	num_observations = 129858
	mean_track_length = 5.83055
	mean_observations_per_image = 3329.69
	mean_reprojection_error = 0.949581
list_num_images = [39]
Reconstruction:
	num_reg_images = 39
	num_cameras = 39
	num_points3D = 22272
	num_observations = 129858
	mean_track_length = 5.83055
	mean_observations_per_image = 3329.69
	mean_reprojection_error = 0.949581
Registered: church / church -> 39 images
Total: church / church -> 41 images
test/church/images/00046.png
test/church/images/00090.png
test/church/images/00092.png
test/church/images/00087.png
test/church/images/00050.png
test/church/images/00068.png
test/church/images/00083.png
test/church/images/00096.png
test/church/images/00069.png
test/church/images/00081.png
test/church/images/00042.png
test/church/images/00018.png
test/church/images/00030.png
test/church/images/00024.png
tes

# Result

In [ ]:
!cat submission.csv